In [2]:
pip install python-docx

  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fpdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ttkthemes

  Using cached ttkthemes-3.2.2.tar.gz (891 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [1]:
pip install language-tool-python

Note: you may need to restart the kernel to use updated packages.


In [12]:
import tkinter as tk
from tkinter import scrolledtext, filedialog, messagebox, ttk
import language_tool_python
from docx import Document
from fpdf import FPDF
import json
import time
import logging
import threading
import queue
import os
import sys
from cryptography.fernet import Fernet
from concurrent.futures import ThreadPoolExecutor
import platform

class GrammarCheckerConfig:
    """Configuration management for the Grammar Checker application."""
    
    def __init__(self):
        self.config_file = "config.json"
        self.default_config = {
            "auto_save_interval": 300,  # seconds
            "default_language": "en-US"
        }
        self.load_config()

    def load_config(self):
        try:
            if os.path.exists(self.config_file):
                with open(self.config_file, 'r') as f:
                    self.config = json.load(f)
            else:
                self.config = self.default_config
                self.save_config()
        except Exception as e:
            logging.error(f"Error loading config: {e}")
            self.config = self.default_config

    def save_config(self):
        try:
            with open(self.config_file, 'w') as f:
                json.dump(self.config, f, indent=4)
        except Exception as e:
            logging.error(f"Error saving config: {e}")

class GrammarCheckerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Professional Grammar Checker")
        self.root.geometry("1200x800")
        
        # Initialize configuration
        self.config = GrammarCheckerConfig()
        
        # Setup logging
        logging.basicConfig(
            filename='grammar_checker.log',
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        
        # Initialize language tool
        self.current_language = self.config.config["default_language"]
        self.tool = language_tool_python.LanguageTool(self.current_language)
        
        # Create GUI elements
        self.create_gui()
        
        # Initialize auto-save using `after()`
        self.auto_save()

    def create_gui(self):
        # Create main container
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create paned window
        self.paned_window = ttk.PanedWindow(self.main_frame, orient=tk.HORIZONTAL)
        self.paned_window.pack(fill=tk.BOTH, expand=True)
        
        # Create left panel (input)
        self.create_input_panel()
        
        # Create right panel (results)
        self.create_results_panel()
        
        # Create status bar
        self.create_status_bar()
        
        # Create menu
        self.create_menu()

    def create_input_panel(self):
        input_frame = ttk.Frame(self.paned_window)
        self.paned_window.add(input_frame, weight=1)
        
        # Text input area
        input_label = ttk.Label(input_frame, text="Enter your text:")
        input_label.pack(pady=5)
        
        self.text_input = scrolledtext.ScrolledText(input_frame, wrap=tk.WORD, height=25)
        self.text_input.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Define tag for highlighting errors
        self.text_input.tag_configure("highlight", background="yellow")

        # Control buttons
        button_frame = ttk.Frame(input_frame)
        button_frame.pack(fill=tk.X, pady=5)
        
        check_button = ttk.Button(button_frame, text="Check Grammar", command=self.check_grammar)
        check_button.pack(side=tk.LEFT, padx=5)
        
        clear_button = ttk.Button(button_frame, text="Clear", command=self.clear_text)
        clear_button.pack(side=tk.LEFT, padx=5)

    def create_results_panel(self):
        results_frame = ttk.Frame(self.paned_window)
        self.paned_window.add(results_frame, weight=1)
        
        results_label = ttk.Label(results_frame, text="Results:")
        results_label.pack(pady=5)
        
        self.results_text = scrolledtext.ScrolledText(results_frame, wrap=tk.WORD, height=25)
        self.results_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

    def create_status_bar(self):
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        self.status_bar = ttk.Label(self.root, textvariable=self.status_var, relief=tk.SUNKEN)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

    def create_menu(self):
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
        
        # File menu
        file_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Open", command=self.open_file)
        file_menu.add_command(label="Save", command=self.save_file)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.exit_application)

    def check_grammar(self):
        text = self.text_input.get("1.0", tk.END).strip()
        if not text:
            messagebox.showwarning("Warning", "Please enter some text to check.")
            return
            
        self.status_var.set("Checking grammar...")
        self.root.update()
        
        try:
            matches = self.tool.check(text)
            self.display_results(matches)
            self.highlight_errors(matches)
        except Exception as e:
            logging.error(f"Grammar check error: {str(e)}")
            messagebox.showerror("Error", f"An error occurred while checking grammar: {str(e)}")
        finally:
            self.status_var.set("Ready")

    def display_results(self, matches):
        self.results_text.delete("1.0", tk.END)
        
        if not matches:
            self.results_text.insert(tk.END, "No grammar mistakes found!")
            return
            
        for i, match in enumerate(matches, 1):
            result = f"Error #{i}:\n"
            result += f"Message: {match.message}\n"
            result += f"Context: {match.context}\n"
            result += f"Suggestions: {', '.join(match.replacements[:3])}\n"
            result += "-" * 50 + "\n\n"
            
            self.results_text.insert(tk.END, result)

    def highlight_errors(self, matches):
        """Highlights errors in the text input area."""
        self.text_input.tag_remove("highlight", "1.0", tk.END)  # Clear previous highlights

        for match in matches:
            start_index = f"1.0 + {match.offset} chars"
            end_index = f"1.0 + {match.offset + match.errorLength} chars"
            self.text_input.tag_add("highlight", start_index, end_index)

    def clear_text(self):
        self.text_input.delete("1.0", tk.END)
        self.results_text.delete("1.0", tk.END)
        self.text_input.tag_remove("highlight", "1.0", tk.END)  # Remove highlights

    def open_file(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        if file_path:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    self.text_input.delete("1.0", tk.END)
                    self.text_input.insert(tk.END, file.read())
            except Exception as e:
                messagebox.showerror("Error", f"Could not open file: {str(e)}")

    def save_file(self):
        file_path = filedialog.asksaveasfilename(
            defaultextension=".txt",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        if file_path:
            try:
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(self.text_input.get("1.0", tk.END))
            except Exception as e:
                messagebox.showerror("Error", f"Could not save file: {str(e)}")

    def exit_application(self):
        confirm = messagebox.askyesno("Exit", "Are you sure you want to exit?")
        if confirm:
            self.root.destroy()

    def auto_save(self):
        self.root.after(300000, self.auto_save)  # Run every 5 minutes

def main():
    root = tk.Tk()
    app = GrammarCheckerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: main thread is not in main loop
Auto-save error: mai